In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from libpysal.weights import Queen
import esda
from esda.moran import Moran, Moran_Local
import contextily as ctx
from splot.esda import moran_scatterplot, plot_moran, lisa_cluster,plot_moran_simulation
import libpysal as lps
import matplotlib.pyplot as plt
import plotly.express as px
from shapely import wkt
from shapely.wkt import loads
import branca.colormap as cm
import folium

df = pd.read_csv("merged_MAR16.csv")

gdf = gpd.GeoDataFrame(df, geometry=df['wkt'].apply(loads))
gdf.set_crs(epsg=3857, inplace=True)

gdf = gdf.dropna()

invalid_geometries = gdf[~gdf.is_valid]
print(f"Found {len(invalid_geometries)} invalid geometries")
gdf['geometry'] = gdf.geometry.buffer(0)

gdf['snap_rate'] = gdf['snap_rate'].astype(float)
gdf['chd_pct'] = gdf['chd_pct'].astype(float)
gdf['lowaccess_pct'] = gdf['lowaccess_pct'].astype(float)
gdf['lowaccess_li_pct'] = gdf['lowaccess_li_pct'].astype(float)
gdf['snap_rate'] = gdf['snap_rate'].astype(float)
gdf['total_pop'] = gdf['total_pop'].astype(int)

gdf['centroid'] = gdf.geometry.centroid
centroids_geo = gdf['centroid'].to_crs(epsg=4326)


gdf['longitude'] = centroids_geo.x
gdf['latitude'] = centroids_geo.y

gdf = gdf[["geometry", "snap_rate", "chd_pct", "lowaccess_pct", "lowaccess_li_pct", "geoid20", "SPA_NAME"]]

In [ ]:
chd_pct_min = gdf['chd_pct'].min()
chd_pct_max = gdf['chd_pct'].max()

snap_rate_min = gdf['snap_rate'].min()
snap_rate_max = gdf['snap_rate'].max()

lowaccess_li_pct_min = gdf['lowaccess_li_pct'].min()
lowaccess_li_pct_max = gdf['lowaccess_li_pct'].max()

lowaccess_pct_min = gdf['lowaccess_pct'].min()
lowaccess_pct_max = gdf['lowaccess_pct'].max()

def style_function_snap_rate(feature):
    return {
        'fillColor': colormap_snap_rate(feature['properties']['snap_rate']),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

def style_function_chd_pct(feature):
    return {
        'fillColor': colormap_chd_pct(feature['properties']['chd_pct']),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

def style_function_lowaccess_li_pct(feature):
    return {
        'fillColor': colormap_chd_pct(feature['properties']['lowaccess_li_pct']),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

def style_function_lowaccess_pct(feature):
    return {
        'fillColor': colormap_chd_pct(feature['properties']['lowaccess_pct']),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

colormap_snap_rate = cm.linear.YlOrRd_09.scale(snap_rate_min, snap_rate_max).to_step(n=5)
colormap_chd_pct = cm.linear.Blues_09.scale(chd_pct_min, chd_pct_max).to_step(n=5)
colormap_lowaccess_li_pct = cm.linear.Purples_09.scale(lowaccess_li_pct_min, lowaccess_li_pct_max).to_step(n=5)
colormap_lowaccess_pct = cm.linear.Oranges_09.scale(lowaccess_pct_min, lowaccess_pct_max).to_step(n=5)

colormap_snap_rate.caption = "SNAP Rate"
colormap_chd_pct.caption = "CHD Percentage"
colormap_lowaccess_li_pct.caption = "Low Access Low Income Percentage"
colormap_lowaccess_pct.caption = "Low Access Percentage"

bounds = gdf.total_bounds

center_lat = (bounds[1] + bounds[3]) / 2
center_lon = (bounds[0] + bounds[2]) / 2

m = folium.Map(location=[center_lat, center_lon], zoom_start=8)

geojson_data = gdf.to_json()

snap_rate_layer = folium.GeoJson(
    geojson_data,
    style_function=style_function_snap_rate,
    name='SNAP Rate',
    popup=folium.GeoJsonPopup(fields=['snap_rate', 'chd_pct', 'lowaccess_li_pct', 'lowaccess_pct'], 
                              aliases=['SNAP Rate:', 'CHD %:', 'Low Income Low Access %', 'Low Access %'], 
                              labels=True)
).add_to(m)

chd_pct_layer = folium.GeoJson(
    geojson_data,
    style_function=style_function_chd_pct,
    name='CHD Percentage',
    popup=folium.GeoJsonPopup(fields=['snap_rate', 'chd_pct', 'lowaccess_li_pct', 'lowaccess_pct'], 
                              aliases=['SNAP Rate:', 'CHD %:', 'Low Income Low Access %', 'Low Access %'], 
                              labels=True)
).add_to(m)

lowaccess_li_pct_layer = folium.GeoJson(
    geojson_data,
    style_function=style_function_lowaccess_li_pct,
    name='Low Income Low Access Percentage',
    popup=folium.GeoJsonPopup(fields=['snap_rate', 'chd_pct', 'lowaccess_li_pct', 'lowaccess_pct'], 
                              aliases=['SNAP Rate:', 'CHD %:', 'Low Income Low Access %', 'Low Access %'], 
                              labels=True)
).add_to(m)

lowaccess_pct_layer = folium.GeoJson(
    geojson_data,
    style_function=style_function_lowaccess_pct,
    name='Low Access Percentage',
    popup=folium.GeoJsonPopup(fields=['snap_rate', 'chd_pct', 'lowaccess_li_pct', 'lowaccess_pct'], 
                              aliases=['SNAP Rate:', 'CHD %:', 'Low Income Low Access %', 'Low Access %'], 
                              labels=True)
).add_to(m)

colormap_snap_rate.add_to(m)
colormap_chd_pct.add_to(m)
colormap_lowaccess_li_pct.add_to(m)
colormap_lowaccess_pct.add_to(m)

folium.LayerControl().add_to(m)

m